In [230]:
import pandas as pd
import numpy as np
import os
import re

Metaphlan is a tool used for taxonomic profiling of metagenomic data. This notebook outlines the steps needed to convert metaphlan output into separate csv files compatible with R's phyloseq. https://github.com/biobakery/MetaPhlAn/wiki/MetaPhlAn-3.0

## To create a phyloseq object you need 3 tables:

### 1. Abundance table

| Otu | Sample_1 | Sample_2 |
|-----|----------|----------|
|Out1 |12461     | 463      |
|Out2 |470       | 73       |

### 2. Taxonomy table

| Otu | Kingdom   | Phylum                | Class                 |  Order                | Family               | Genus               | Species                        |
|-----|-----------|-----------------------|-----------------------|-----------------------|----------------------|---------------------|--------------------------------|
|Out1 |k__Viruses |p__Viruses_unclassified|c__Viruses_unclassified|o__Viruses_unclassified|f__Polydnaviridae     |g__Bracovirus        |s__Cotesia_congregata_bracovirus|                             
|Out2 |k__Archaea |p__Euryarchaeota       |c__Methanobacteria     |o__Methanobacteriales  |f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii   |        

### 3. Sample table

| Otu     | Condition  | Treatment |
|---------|------------|-----------|
|Sample_1 |CD          | 1         |
|Sample_2 |UC          | 2         |

### Sample metaphlan output

Metaphlan gives us output in a compressed format by putting all the taxa identified in the ID column and the quantity of taxa present in each sample column. The best way to approach this dataset is to extract only the rows of the taxa you wish to look at. For example, if you want to look at the species level filter out all the rows which end in "s__" or for order filter the rows ending in "o__".

|ID                                            |    Sample_1_profiled_metagenome | Sample_2_profiled_metagenome|
|----------------------------------------------|---------------------------------|-----------------------------|
|UNKNOWN                                                             |   92.02399| 91.02948                    |
|k__Archaea                                                          |   0.052938| 0.045632                    |
|k__Archaea\p__Euryarhaeota                                          |   0.052938|0.045632                     |
|k__Archaea\p__Euryachaeota\c__Methanobacteria                       |   0.052938|0.045632                     |
|k__Archaea\p__Euryarchaeota\c__Methanobacteria\o__Methanobacteriales|   0.052938|0.045632                     | 

### Helper functions

In [224]:
def clean_merged_table(df,merged_file_name):
    '''Cleans the merged table by removing file extenstion suffix which gets added to sample columns during merge.'''
    #df = pd.read_table(df,sep='\t',engine='python')
    df.columns = df.columns.str.replace(merged_file_name, '')
    return df

def get_taxa_columns(df,rank):
    '''Splits ID into taxanomic ranks to make taxa table''' 
    df_taxa = df['ID'].str.split('|',expand=True)
    taxa_cols = ["Kingdom","Phylum","Class","Order","Family","Genus","Species","Strain"]
    taxa_dict = {'Kingdom':1,"Phylum":2,"Class":3,"Order":4,"Family":5,"Genus":6,"Species":7,"Strain":8}
    value = taxa_dict.get(rank)
    taxa_cols=taxa_cols[0:value]
    df_taxa.columns=taxa_cols
    for col in df_taxa.columns:
        df_taxa[col]=df_taxa[col].apply(trim_taxa_names)    
    otu_index = []
    for i in range(0, len(df)):
        otu_index.append("Otu"+str(i))
    df_taxa['Otu']=otu_index 
    taxa_cols=[col for col in df_taxa.columns if 'Otu' not in col]
    for col in taxa_cols:
        df_taxa.at[df_taxa.index[-1], col] = 'Other'
    return df_taxa

def trim_taxa_names(x):
    '''Removes leading characters before taxa ID e.g. s__ '''
    match = re.sub(r'^[kpcofgs]__',"",str(x))
    return match

def get_sample_cols(df):
    '''Finds and returns sample columns in dataframe, presuming sample names contain a number'''
    r = re.compile(r'^.*[0-9].*$') #match column names that contain a number anywhere
    sample_cols=[]
    for col in df:
        if(r.match(col)):
            sample_cols.append(col)
    return sample_cols

def create_sample_df(abun_matrix):
    sample_cols=get_sample_cols(abun_matrix)
    sample_df=pd.DataFrame({'Sample':sample_cols})
    sample_df['Behaviour'] = sample_df['Sample'].apply(get_behaviour)
    # Add extra column to sample df so phyloseq ordination plots behave
    sample_df['Type'] = 'murine'
    return sample_df

def add_otu_primary_key(df):
    '''Adds otu primary key column to dataframe'''
    otu_index = []
    for i in range(0, len(df)):
        otu_index.append("Otu"+str(i))
    df['Otu']=otu_index 
    return df

def get_behaviour(x):
    '''Creates sample_data columns based on sample name starting with letter'''
    if x.startswith('E'):
        return "Extinction"
    elif x.startswith('R'):
        return "Resilient"
    elif x.startswith('S'):
        return "Susceptible"
    else:
        return ''
    
def create_other_group(df,thresh):
    '''From a given threshold or cut-off point it creates an "Other" row with the summed abundance of 
    species below the givedn threshold. Used to give idea of remainder of species excluded from plot legend'''
    index_to_append = len(df)
    df.at[index_to_append,'ID']= 'Other'
    cols= df.columns 
    for col in cols:
        if col!='ID':
            df.loc[df['ID']=='Other', col]= np.sum(df.loc[(df[col]<=thresh) & (df['ID']!="other")][col])
    sample_cols = get_sample_cols(df)
    ids=df['ID'].to_list()
    df_thresh_removed = df[sample_cols].apply(lambda x: np.where(x <= thresh,0,x))
    df_thresh_removed['ID']=ids
    df=df_thresh_removed[(df_thresh_removed.iloc[:,0:-1] > 0).any(axis=1)]
    df.reset_index(drop=True,inplace=True)
    return df

def create_sample_df(abun_matrix):
    sample_cols=get_sample_cols(abun_matrix)
    sample_df=pd.DataFrame({'Sample':sample_cols})
    sample_df['Behaviour'] = sample_df['Sample'].apply(get_behaviour)
    # Add extra column to sample df so phyloseq ordination plots behave
    sample_df['Type'] = 'murine'
    return sample_df

## Importing the data

In [186]:
os.getcwd()

'/home/linda/Projects/metaphlan'

In [187]:
!ls /home/linda/Projects/metaphlan/data

pool1_profile_known.txt  sample_df.csv		      species_taxa.csv
pool2_profile_known.txt  species_known_abundance.csv


I have called these files profile "known" since the relative abundances are calculated out of the proportion of known reads that were classified. It is possible to estimate the relative abundances including the unknown fraction of reads by running metaphlan with the parameter --unknown-estimation. 

There are 2 files from 2 different sequencing pools which I am going to merge. These filed were created by metaphlan's merge_metaphlan_tables.py file.

In [188]:
df_k_1 = pd.read_table("data/pool1_profile_known.txt", sep='\t')
df_k_2 = pd.read_table("data/pool2_profile_known.txt", sep='\t')

In [189]:
df_k_1.head()

,ID,E15_S89_known_profiled_metagenome,E19b_S88_known_profiled_metagenome,E28_S87_known_profiled_metagenome,E5b_S84_known_profiled_metagenome,E7_S78_known_profiled_metagenome,E8_S74_known_profiled_metagenome,R21b_S76_known_profiled_metagenome,R24_S82_known_profiled_metagenome,R5_S75_known_profiled_metagenome,R6_S81_known_profiled_metagenome,S11b_S73_known_profiled_metagenome,S11_S83_known_profiled_metagenome,S13_S79_known_profiled_metagenome,S22b_S90_known_profiled_metagenome,S22_S77_known_profiled_metagenome,S3b_S86_known_profiled_metagenome,S4_S85_known_profiled_metagenome,S9_S80_known_profiled_metagenome
0,k__Bacteria,97.65801,99.37595,96.86904,93.72727,97.30751,96.97202,99.28079,99.51966,97.72520,99.77435,99.13651,97.13792,99.71291,99.81773,98.82742,96.79303,96.87159,98.24866
1,k__Bacteria|p__Actinobacteria,1.77270,35.18209,0.93580,1.73347,1.61261,1.83636,0.89332,1.38952,1.59392,2.32794,1.11673,2.29589,1.62908,21.00678,0.84331,11.85350,0.99145,2.12132
2,k__Bacteria|p__Actinobacteria|c__Actinobacteria,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,20.50706,0.00000,11.21542,0.00000,0.00000
3,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,20.50706,0.00000,11.21542,0.00000,0.00000
4,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,20.50706,0.00000,11.21542,0.00000,0.00000


### Lets look at genus level by using a regular expression to filter rows based on last occurrence of a letter

In [190]:
df_k_1_genus=df_k_1[df_k_1['ID'].str.contains(r'\|g__[^|]*$')]
df_k_2_genus=df_k_2[df_k_2['ID'].str.contains(r'\|g__[^|]*$')]
df_k_1_genus.reset_index(drop=True,inplace=True)
df_k_2_genus.reset_index(drop=True,inplace=True)

In [191]:
# Here we have only the rows corresponding to genus level
df_k_1_genus.head()

,ID,E15_S89_known_profiled_metagenome,E19b_S88_known_profiled_metagenome,E28_S87_known_profiled_metagenome,E5b_S84_known_profiled_metagenome,E7_S78_known_profiled_metagenome,E8_S74_known_profiled_metagenome,R21b_S76_known_profiled_metagenome,R24_S82_known_profiled_metagenome,R5_S75_known_profiled_metagenome,R6_S81_known_profiled_metagenome,S11b_S73_known_profiled_metagenome,S11_S83_known_profiled_metagenome,S13_S79_known_profiled_metagenome,S22b_S90_known_profiled_metagenome,S22_S77_known_profiled_metagenome,S3b_S86_known_profiled_metagenome,S4_S85_known_profiled_metagenome,S9_S80_known_profiled_metagenome
0,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae|g__Bifidobacterium,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,20.50706,0.00000,11.21542,0.00000,0.00000
1,k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Eggerthellales|f__Eggerthellaceae|g__Adlercreutzia,0.01522,0.02269,0.00986,0.01337,0.02927,0.04169,0.01290,0.00000,0.02503,0.05092,0.01569,0.03171,0.04299,0.01102,0.02144,0.01528,0.01784,0.03676
2,k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Eggerthellales|f__Eggerthellaceae|g__Asaccharobacter,0.06825,0.02827,0.02306,0.02648,0.02673,0.03658,0.02223,0.00543,0.03252,0.06713,0.02811,0.04405,0.03960,0.01511,0.00854,0.01129,0.01096,0.06209
3,k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Eggerthellales|f__Eggerthellaceae|g__Enterorhabdus,1.68923,1.13757,0.90287,1.69362,1.55661,1.75809,0.85820,1.38410,1.53637,2.20989,1.07292,2.22013,1.54649,0.47359,0.81333,0.61152,0.96265,2.02247
4,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides,2.57402,1.03313,2.58441,5.52997,4.55413,2.52983,1.46428,2.77514,8.90767,1.47206,0.87271,4.22791,0.60317,0.46695,2.50889,4.48199,6.83279,1.09368


In [192]:
df_k_1_genus.shape

(38, 19)

In [193]:
# Merging the two pools on ID
df_known_genus = pd.merge(df_k_1_genus,df_k_2_genus,on=['ID'],how='outer')

In [194]:
df_known_genus.shape

(38, 38)

## Clean the ID column and sample name columns to create the abundance matrix

In [195]:
df_known=clean_merged_table(df_known_genus,'_known_profiled_metagenome')
df_known[0:2]

,ID,E15_S89,E19b_S88,E28_S87,E5b_S84,E7_S78,E8_S74,R21b_S76,R24_S82,R5_S75,...,R14_S71,R21_S60,R26_S67,R29_S61,R2_S58,R6b_S54,R7b_S68,S20b_S59,S27_S72,S2b_S62
0,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae|g__Bifidobacterium,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.0000,0.0,0.00000,...,0.00000,0.00000,70.28168,0.00000,0.00000,0.00000,0.00000,31.72024,0.00000,10.61550
1,k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Eggerthellales|f__Eggerthellaceae|g__Adlercreutzia,0.01522,0.02269,0.00986,0.01337,0.02927,0.04169,0.0129,0.0,0.02503,...,0.04023,0.04791,0.01381,0.04743,0.04462,0.00712,0.02831,0.06023,0.01183,0.02226


In [196]:
df_known.tail()

,ID,E15_S89,E19b_S88,E28_S87,E5b_S84,E7_S78,E8_S74,R21b_S76,R24_S82,R5_S75,...,R14_S71,R21_S60,R26_S67,R29_S61,R2_S58,R6b_S54,R7b_S68,S20b_S59,S27_S72,S2b_S62
33,k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Herpesvirales|f__Herpesviridae|g__Rhadinovirus,0.00000,0.00000,0.10895,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.11163
34,k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Ortervirales|f__Retroviridae|g__Betaretrovirus,0.23658,0.10169,0.09105,0.43274,0.28599,0.05585,0.03707,0.00000,0.13411,...,0.06558,0.00000,0.07879,0.22339,0.12315,0.04778,0.06023,0.02109,0.66410,1.23608
35,k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Ortervirales|f__Retroviridae|g__Gammaretrovirus,0.71291,0.06184,0.28809,1.54332,0.65427,0.37626,0.28503,0.06873,0.18652,...,0.13752,0.20025,0.19943,0.47122,0.38966,0.20041,0.88896,0.64872,1.68495,8.66917
36,k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Viruses_unclassified|f__Baculoviridae|g__Alphabaculovirus,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.03318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Viruses_unclassified|f__Polydnaviridae|g__Bracovirus,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.08873,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.11210


In [197]:
# Replace NaN with 0
df_known.fillna(0,inplace=True)

In [198]:
id_column = df_known[['ID']]

In [200]:
abund_matrix=create_other_group(df_known,3) # Setting 3 as threshold - anything taxa comprising 3% or less of the matrix will be grouped into other
abund_matrix

,E15_S89,E19b_S88,E28_S87,E5b_S84,E7_S78,E8_S74,R21b_S76,R24_S82,R5_S75,R6_S81,...,R21_S60,R26_S67,R29_S61,R2_S58,R6b_S54,R7b_S68,S20b_S59,S27_S72,S2b_S62,ID
0,0.00000,33.99356,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,70.28168,0.00000,0.00000,0.00000,0.00000,31.72024,0.00000,10.61550,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae|g__Bifidobacterium
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,3.44706,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Eggerthellales|f__Eggerthellaceae|g__Enterorhabdus
2,0.00000,0.00000,0.00000,5.52997,4.55413,0.00000,0.00000,0.00000,8.90767,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,4.36496,0.00000,0.00000,3.20713,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides
3,32.67238,15.48173,19.72849,29.41141,31.89512,37.51055,5.50114,12.37102,38.94397,47.76264,...,16.31096,6.75982,32.03713,14.61179,10.52932,15.37850,17.26982,22.22244,26.37850,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Muribaculaceae|g__Muribaculaceae_unclassified
4,25.68020,9.66215,15.47967,25.57575,26.25145,14.94594,10.05876,6.31194,19.81598,7.73166,...,31.06341,8.38871,29.84066,10.03842,9.99244,11.13738,6.07923,39.77292,10.81153,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Muribaculaceae|g__Muribaculum
5,4.98770,0.00000,6.95289,0.00000,0.00000,0.00000,22.89472,3.14556,0.00000,0.00000,...,3.29901,0.00000,0.00000,3.62847,20.39355,23.80294,15.56993,4.02813,0.00000,k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella
6,11.98444,6.50151,7.27066,4.77874,7.06418,6.39057,7.94512,6.45230,11.26780,15.46397,...,17.59737,0.00000,7.57646,5.05002,3.38600,3.54435,5.69944,3.89995,3.61393,k__Bacteria|p__Bacteroidetes|c__Flavobacteriia|o__Flavobacteriales|f__Flavobacteriaceae|g__Imtechella
7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,14.32023,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,10.29383,0.00000,0.00000,0.00000,0.00000,k__Bacteria|p__Deferribacteres|c__Deferribacteres|o__Deferribacterales|f__Deferribacteraceae|g__Mucispirillum
8,6.56370,3.93777,12.25170,3.03387,13.31346,6.56068,16.78792,40.71146,11.46063,0.00000,...,15.52980,3.70173,10.77961,7.31977,20.50488,32.77265,5.87253,8.71473,0.00000,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus
9,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium


In [201]:
abun_matrix=add_otu_primary_key(df_known)

In [203]:
abun_matrix[0:1]

,ID,E15_S89,E19b_S88,E28_S87,E5b_S84,E7_S78,E8_S74,R21b_S76,R24_S82,R5_S75,...,R21_S60,R26_S67,R29_S61,R2_S58,R6b_S54,R7b_S68,S20b_S59,S27_S72,S2b_S62,Otu
0,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Bifidobacteriales|f__Bifidobacteriaceae|g__Bifidobacterium,0.0,33.99356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,70.28168,0.0,0.0,0.0,0.0,31.72024,0.0,10.6155,Otu0


## Create taxa table

In [220]:
genus_taxa=get_taxa_columns(abun_matrix,'Genus')
genus_taxa.tail()

,Kingdom,Phylum,Class,Order,Family,Genus,Otu
34,Viruses,Viruses_unclassified,Viruses_unclassified,Ortervirales,Retroviridae,Betaretrovirus,Otu34
35,Viruses,Viruses_unclassified,Viruses_unclassified,Ortervirales,Retroviridae,Gammaretrovirus,Otu35
36,Viruses,Viruses_unclassified,Viruses_unclassified,Viruses_unclassified,Baculoviridae,Alphabaculovirus,Otu36
37,Viruses,Viruses_unclassified,Viruses_unclassified,Viruses_unclassified,Polydnaviridae,Bracovirus,Otu37
38,Other,Other,Other,Other,Other,Other,Otu38


## Create sample data

In [229]:
# Extracts all columns containing a number in name
sample_cols=get_sample_cols(abun_matrix)
sample_cols[0:7]

['E15_S89', 'E19b_S88', 'E28_S87', 'E5b_S84', 'E7_S78', 'E8_S74', 'R21b_S76']

In [227]:
sample_df=create_sample_df(abund_matrix)
sample_df.head()

,Sample,Behaviour,Type
0,E15_S89,Extinction,murine
1,E19b_S88,Extinction,murine
2,E28_S87,Extinction,murine
3,E5b_S84,Extinction,murine
4,E7_S78,Extinction,murine


## Export to csv for importing into phyloseq

In [ ]:
abund_matrix.to_csv('data/genus_abundance.csv',index=False)
genus_taxa.to_csv('data/genus_taxa.csv',index=False)
sample_df.to_csv('data/sample_data.csv',index=False)